<a href="https://colab.research.google.com/github/opedoussaut/orbit/blob/main/ORBIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ORBIT
## Structuring circular life-cycle knowledge with ontologies  
### Version: v0.1.0
> Minimal ORIONT-aligned structure with basic SHACL completeness and consistency checks


This notebook explores how circular life-cycle knowledge can be formally structured using ontologies.
The focus is on making key concepts, boundaries, and relationships explicit and inspectable,
rather than on computing sustainability indicators.

## Scope

This notebook:
- builds a minimal ORIONT-aligned graph (ProductSystem, Activity, LCStage, LCIAResult)
- adds SHACL shapes for basic completeness/consistency checks
- demonstrates validation on a synthetic example

This notebook does not:
- compute LCA/LCSA indicators
- propose a new methodology
- implement a production system

## Conceptual view

Methodology
   ↓
Ontology (ORIONT)
   ↓
Structured life-cycle knowledge
   ↓
Inspection, validation, and discussion


> The ontology is used here as a structuring layer, not as a computation engine.



In [ ]:
!pip -q install rdflib pyshacl

In [ ]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD

# Namespaces
ORBIT = Namespace("http://example.org/orbit#")

def u(local_name: str) -> URIRef:
    """Convenience helper to create URIs in the ORBIT namespace."""
    return ORBIT[local_name]

In [ ]:
g = Graph()
g.bind("orbit", ORBIT)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("xsd", XSD)

# --- Classes (ORIONT-aligned names) ---
for cls in ["ProductSystem", "Activity", "LCStage", "LCIAResult", "IndicatorUnit"]:
    g.add((u(cls), RDF.type, RDFS.Class))

# --- Properties ---
# Object properties
g.add((u("hasActivity"), RDF.type, RDF.Property))
g.add((u("hasActivity"), RDFS.domain, u("ProductSystem")))
g.add((u("hasActivity"), RDFS.range,  u("Activity")))

g.add((u("hasLCStage"), RDF.type, RDF.Property))
g.add((u("hasLCStage"), RDFS.domain, u("Activity")))
g.add((u("hasLCStage"), RDFS.range,  u("LCStage")))

g.add((u("hasLCIAResult"), RDF.type, RDF.Property))
g.add((u("hasLCIAResult"), RDFS.domain, u("Activity")))
g.add((u("hasLCIAResult"), RDFS.range,  u("LCIAResult")))

g.add((u("hasUnit"), RDF.type, RDF.Property))
g.add((u("hasUnit"), RDFS.domain, u("LCIAResult")))
g.add((u("hasUnit"), RDFS.range,  u("IndicatorUnit")))

# Data properties (ORIONT-inspired)
g.add((u("hasReferenceYear"), RDF.type, RDF.Property))
g.add((u("hasReferenceYear"), RDFS.domain, u("Activity")))
g.add((u("hasReferenceYear"), RDFS.range,  XSD.gYear))

g.add((u("hasValidityYear"), RDF.type, RDF.Property))
g.add((u("hasValidityYear"), RDFS.domain, u("Activity")))
g.add((u("hasValidityYear"), RDFS.range,  XSD.gYear))

g.add((u("hasNumericalValue"), RDF.type, RDF.Property))
g.add((u("hasNumericalValue"), RDFS.domain, u("LCIAResult")))
g.add((u("hasNumericalValue"), RDFS.range,  XSD.decimal))

g.add((u("indicatorName"), RDF.type, RDF.Property))
g.add((u("indicatorName"), RDFS.domain, u("LCIAResult")))
g.add((u("indicatorName"), RDFS.range,  XSD.string))

print("Schema triples:", len(g))

In [ ]:
# LC stages (synthetic but realistic)
for stage, label in [
    ("Manufacturing", "Manufacturing"),
    ("Use", "Use"),
    ("EndOfLife", "End-of-life")
]:
    g.add((u(stage), RDF.type, u("LCStage")))
    g.add((u(stage), RDFS.label, Literal(label)))

# Example indicator unit
g.add((u("kgCO2eq"), RDF.type, u("IndicatorUnit")))
g.add((u("kgCO2eq"), RDFS.label, Literal("kg CO2 eq")))

print("After individuals:", len(g))

In [ ]:
PS1 = u("PS_1")
A1  = u("A1_Manufacturing")
A2  = u("A2_Use")
R1  = u("R1_ClimateChange")

# Type assertions
g.add((PS1, RDF.type, u("ProductSystem")))
g.add((A1,  RDF.type, u("Activity")))
g.add((A2,  RDF.type, u("Activity")))
g.add((R1,  RDF.type, u("LCIAResult")))

# Link product system → activities
g.add((PS1, u("hasActivity"), A1))
g.add((PS1, u("hasActivity"), A2))

# Assign stages to activities
g.add((A1, u("hasLCStage"), u("Manufacturing")))
g.add((A2, u("hasLCStage"), u("Use")))

# Temporal scope (gYear)
g.add((A1, u("hasReferenceYear"), Literal("2024", datatype=XSD.gYear)))
g.add((A1, u("hasValidityYear"),  Literal("2025", datatype=XSD.gYear)))
g.add((A2, u("hasReferenceYear"), Literal("2024", datatype=XSD.gYear)))
g.add((A2, u("hasValidityYear"),  Literal("2025", datatype=XSD.gYear)))

# LCIA result attached to A1
g.add((A1, u("hasLCIAResult"), R1))
g.add((R1, u("indicatorName"), Literal("Climate change")))
g.add((R1, u("hasNumericalValue"), Literal("12.34", datatype=XSD.decimal)))
g.add((R1, u("hasUnit"), u("kgCO2eq")))

print("Total triples:", len(g))

In [ ]:
print("Activities in PS_1:")
for act in g.objects(PS1, u("hasActivity")):
    stage = next(g.objects(act, u("hasLCStage")), None)
    print("-", act, "stage:", stage)

## Validation (SHACL)

OWL/RDF expresses meaning and structure under an open-world assumption.
SHACL is used here to express minimal closed-world checks (completeness and consistency).

In [ ]:
shapes_ttl = """
@prefix sh:    <http://www.w3.org/ns/shacl#> .
@prefix xsd:   <http://www.w3.org/2001/XMLSchema#> .
@prefix orbit: <http://example.org/orbit#> .

orbit:ActivityShape a sh:NodeShape ;
  sh:targetClass orbit:Activity ;
  sh:property [
    sh:path orbit:hasLCStage ;
    sh:minCount 1 ;
    sh:maxCount 1 ;
    sh:class orbit:LCStage ;
    sh:message "Activity must have exactly one LCStage." ;
  ] ;
  sh:property [
    sh:path orbit:hasReferenceYear ;
    sh:minCount 1 ;
    sh:datatype xsd:gYear ;
    sh:message "Activity must have hasReferenceYear (xsd:gYear)." ;
  ] ;
  sh:property [
    sh:path orbit:hasValidityYear ;
    sh:minCount 1 ;
    sh:datatype xsd:gYear ;
    sh:message "Activity must have hasValidityYear (xsd:gYear)." ;
  ] ;
  sh:sparql [
    a sh:SPARQLConstraint ;
    sh:message "Validity year must be >= reference year." ;
    sh:select '''
      SELECT $this WHERE {
        $this orbit:hasReferenceYear ?ry ;
              orbit:hasValidityYear  ?vy .
        FILTER (?vy < ?ry)
      }
    ''' ;
  ] .

orbit:LCIAResultShape a sh:NodeShape ;
  sh:targetClass orbit:LCIAResult ;
  sh:property [
    sh:path orbit:hasNumericalValue ;
    sh:minCount 1 ;
    sh:datatype xsd:decimal ;
    sh:message "LCIAResult must have a numerical value (xsd:decimal)." ;
  ] ;
  sh:property [
    sh:path orbit:hasUnit ;
    sh:minCount 1 ;
    sh:class orbit:IndicatorUnit ;
    sh:message "LCIAResult must have a unit (IndicatorUnit)." ;
  ] .
"""

In [ ]:
from pyshacl import validate
from rdflib import Graph

shapes_g = Graph()
shapes_g.parse(data=shapes_ttl, format="turtle")

conforms, report_graph, report_text = validate(
    data_graph=g,
    shacl_graph=shapes_g,
    inference="rdfs",
    abort_on_first=False
)

print("CONFORMS:", conforms)
print(report_text)

In [ ]:
# Remove LCStage from A2 to trigger a failure
for o in list(g.objects(A2, u("hasLCStage"))):
    g.remove((A2, u("hasLCStage"), o))

conforms2, _, report_text2 = validate(data_graph=g, shacl_graph=shapes_g)
print("CONFORMS after breaking:", conforms2)
print(report_text2)

In [ ]:
ttl_out = g.serialize(format="turtle")
print(ttl_out[:1500])  # preview only